<a href="https://colab.research.google.com/github/MaxCojocari/Py_labs/blob/main/probl3_MD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3. Matrix**

You have a set of 20 people connected via a friendship matrix. The whole list is given in
*matrix.txt.*.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **3.1 Friends**
Find the person with the most friends.

In [ ]:
import re
from collections import deque

In [ ]:
#Create a list for vertices
def read_vertices(fl = open('/content/drive/MyDrive/matematica/matrix.txt', 'r'), 
                  vertices = []):
    first_line = fl.readline()
    for x in re.split('\t|\n| \| ', first_line): 
        #after splitting we have also in list empty strings (delimited by \t)
        #we select only names
        if x!='':
            vertices.append(x)
    fl.close()
    return vertices

vertices = read_vertices()


In [ ]:
def read_rel_matrix(fl = open('/content/drive/MyDrive/matematica/matrix.txt', 'r'), 
                    raw = [], rel_matrix = []):
    for i, f in enumerate(fl):
        #if first line, pass
        if i==0:
            continue
        else:
            #extract boolean values, line by line
            line_txt = f.split()
            for x in line_txt:
                if x=='0' or x=='1':
                    raw.append(int(x))
            rel_matrix.append(raw)
            raw = []
    fl.close()
    return rel_matrix

rel_matrix = read_rel_matrix()

In [ ]:
#Read graph's edges
def read_edges(rel_matrix, edges = []):
    for i in range(20):
        for j in range(20):
            if rel_matrix[i][j]==1:
                edges.append((vertices[i], vertices[j]))
    return edges

edges = read_edges(rel_matrix)

In [ ]:
#Count nr of friends for each person:
def count_friends(edges, nr_friends={}):
    for pair in edges:
        if pair[0] not in nr_friends:
            nr_friends.update({pair[0] : 1})
        else:
            nr_friends[pair[0]] += 1
    return nr_friends

nr_friends = count_friends(edges)

In [ ]:
#Sort people by nr of friends
def sort_by_nr_friends(nr_friends):
    return dict(sorted(nr_friends.items(), key = lambda x: x[1], reverse = True))

In [ ]:
nr_friends = sort_by_nr_friends(nr_friends)

print(f'Person with most friends:') 
print(f'{list(nr_friends.keys())[0]}')

Person with most friends:
Corrin Tally


# **3.2 Sort**
Sort all the people by the number of friends.

In [ ]:
print('People sorted by the number of friends:')
for key, value in nr_friends.items():
    if len(key) < 16:
        print("{}\t\t{}".format(key, value))
    else:
        print("{}\t{}".format(key, value))

People sorted by the number of friends:
Corrin Tally		11
Ellie Francese		11
Augustine Golub		10
Caleb Hobby		7
Leandro Eagan		7
Clarence Stalker	6
Lili Houghton		6
Sammie Womac		6
Pearlie Moffet		6
Cruz Perna		6
Lorean Simcox		6
Monet Mccoy		5
Angila Ellinger		5
Marita Tegeler		5
Tiny Parkhurst		4
Alta Kennan		2
Otilia Laxson		2
Elinore Orsborn		2
Rebbecca Charlton	2
Jarred Marrow		1


# **3.3 Let's do ratings**
How to do that? Well, each person in the graph is connected to everyone else at some level.
Therefore, each person will have a list of connections which is as long as the total list of
people in the graph (in our case, 20). You then have to compute the shortest path from each
of the nodes to each of the other nodes.
For example, let’s say that you found that from node 0 you can reach node 3 in 5 steps (that
is, the shortest path connecting nodes 0 and 3 has 5 steps). That means that node 3 will be
a connection of level 5 to node 0 and will therefore contribute to 0 with 4 points.

As a procedure, you can take each item n and then compute the distances between *n* and all
the other vertices of the graph. You can use these distances to compute the value that is
added by each of the other *n* − 1 vertices to *n*. Sum it and you’ll have the value of vertex *n*.

In order to find the shortest path between two vertices, you’ll have to use Dijkstra's algorithm.

You can find plenty of implementations of that algorithm online.
Compute the points for each person in our network. Let’s call it ”Rating”

In [ ]:
#Create a graph in form of dictionary from previously read edges
#G = {vertex1:{neighbour_vertex1: 1, neighbour_vertex2: 1, ...}, vertex2:{...}, ...}
def create_relationship_graph(edges, G = {}):
    for edge in edges:
        if edge[0] not in G:
            G[edge[0]] = {edge[1]: 1}
        else:
            G[edge[0]][edge[1]] = 1
    return G

G = create_relationship_graph(edges)

In [ ]:
#Dijkstra algorithm implementation which outputs the shortest distances
#between start vertex and other ones
def dijkstra(G, start):
    Q = deque()
    s = {}
    s[start] = 0
    Q.append(start)
    while Q:
        v = Q.popleft()
        for u in G[v]:
            if u not in s or s[v] + G[v][u] < s[u]:
                s[u] = s[v] + G[v][u]
                Q.append(u)
    return s

In [ ]:
#Find rating for each person 
def find_rating(G, vertices, rating = {}):
    for person in vertices:
        edge_separate_ratings = dijkstra(G, person)
        total_rating = 0
        for value in edge_separate_ratings.values():
            total_rating += value
        rating.update({person: total_rating})
    return rating


In [ ]:
rating = find_rating(G, vertices)

print("Rating for all people:")
for key, value in rating.items():
    if len(key) < 16:
        print("{}\t\t{}".format(key, value))
    else:
        print("{}\t{}".format(key, value))

Rating for all people:
Caleb Hobby		32
Alta Kennan		46
Corrin Tally		30
Leandro Eagan		34
Otilia Laxson		49
Ellie Francese		27
Augustine Golub		31
Elinore Orsborn		49
Clarence Stalker	37
Lili Houghton		36
Monet Mccoy		39
Angila Ellinger		36
Sammie Womac		36
Tiny Parkhurst		40
Pearlie Moffet		32
Cruz Perna		35
Rebbecca Charlton	42
Marita Tegeler		37
Jarred Marrow		50
Lorean Simcox		32


# **3.4 Influential people**
Let’s say that each of these people has a certain rate of posting content. Obviously, people
who communicate more are much more influential. Suppose that you need to promote a new
brand using social media. We found out how often each of these 20 people writes something
on their walls. You can find it in influence.txt

Whom of these people will you contact? Why? Be advised that not only the frequency of
posting matters, but also the number of friends!

Use the data from the previous exercise and find the new ”Rating” for each person by
multiplying it with 0.5 of the posting rate.

Please sort the people by the newly computed rating.

In [ ]:
fl = open('/content/drive/MyDrive/matematica/influence.txt', 'r')
def new_rating(fl, rating):
    for line in fl:
        split_line = re.split(' : |\n', line)
        rating[split_line[0]] *= 0.5*float(split_line[1])
    fl.close()
    return rating

new_brute_rating = new_rating(fl, rating)

In [ ]:
#Sort people by new rating
def sort_by_rating(rating):
    return dict(sorted(rating.items(), key = lambda x: x[1], reverse = True))

In [ ]:
print("People sorted by new rating:")
new_ratings = sort_by_rating(rating)
for key, value in new_ratings.items():
    if len(key) < 16:
        print("{}\t\t{}".format(key, value))
    else:
        print("{}\t{}".format(key, value))

People sorted by new rating:
Alta Kennan		227.70000000000002
Jarred Marrow		191.25
Tiny Parkhurst		169.0
Rebbecca Charlton	161.70000000000002
Sammie Womac		153.0
Angila Ellinger		148.5
Marita Tegeler		139.21249999999998
Corrin Tally		128.62500000000003
Leandro Eagan		113.05000000000001
Otilia Laxson		110.86250000000001
Lorean Simcox		110.39999999999999
Ellie Francese		110.02499999999998
Monet Mccoy		108.22500000000001
Elinore Orsborn		107.80000000000001
Lili Houghton		99.0
Cruz Perna		91.875
Augustine Golub		87.57499999999999
Clarence Stalker	64.2875
Pearlie Moffet		52.400000000000006
Caleb Hobby		44.400000000000006


# **3.5 Analyze your content**
You are publishing a book and would like to promote it through the use of social media. The
book’s title is ”From T-Rex to Multi Universes: How the Internet has Changed Politics, Art
and Cute Cats.” You have done some research in the world’s most popular social network
and have found that the range of interests is stored in interests.txt

Analyze your title and see what specter of interests is your book marketable to.

In [ ]:
fl = open('/content/drive/MyDrive/matematica/interests.txt', 'r')
name_book = 'From T-Rex to Multi Universes: How the Internet has Changed Politics, Art and Cute Cats.'

In [ ]:
#Each word from title is splitted separately
words_title = re.split(',|:| ', name_book)

In [ ]:
def find_similar_interests(fl, words_title, similar_interests = []):
    #Each interest is read from interests.txt
    for line in fl:
        interest = re.split('\n', line)
        #If an interest is found in title, it is encounted
        if interest[0] in words_title:
            similar_interests.append(interest[0])
    fl.close()
    return similar_interests

In [ ]:
specter_of_interests = find_similar_interests(fl, words_title)
print("Similar interests:")
print(*specter_of_interests, sep='\n')

Similar interests:
Internet
T-Rex
Art
Politics


# **3.6 Promote it**
We have provided you with a list of interests of each of these people. You can find it in
interests.txt.

Considering the set of interests you have chosen, who of them would you market the book
to? Let’s say that a person has 5 of her interests coinciding with your books and she has a
Rating of 346. Multiply her rating with the 0.2 * coinciding interests to see a final score. Sort
the people by this final score.

Provide us with a list of 5 people we should contact to make your book a bestseller! Please
use the names found in people_interests.txt.

In [ ]:
fl = open('/content/drive/MyDrive/matematica/people_interests.txt', 'r')

In [ ]:
#Create a dictionary with people's interests from people_interests.txt
def find_people_interests(fl, people_interests = {}):
    for line in fl:
        my_str = re.split(' : |\n', line)
        people_interests.update({my_str[0]: my_str[1].split()})     
    fl.close()
    return people_interests

people_interests = find_people_interests(fl)
print(people_interests)

{'Caleb Hobby': ['Music', 'Bieber', 'Internet', 'Movies'], 'Alta Kennan': ['Programming', 'T-Rex', 'Computers', 'Art'], 'Corrin Tally': ['Computers', 'Books', 'Football'], 'Leandro Eagan': ['Football', 'Cats'], 'Otilia Laxson': ['Internet', 'Videogames'], 'Ellie Francese': ['Programming', 'Movies', 'Cats'], 'Augustine Golub': ['Planes', 'Internet'], 'Elinore Orsborn': ['Cars', 'Internet'], 'Clarence Stalker': ['Movies', 'Planes'], 'Lili Houghton': ['Theatre', 'Computers'], 'Monet Mccoy': ['Internet', 'Theatre', 'Books'], 'Angila Ellinger': ['Books', 'Cats', 'Programming'], 'Sammie Womac': ['Videogames', 'Bieber', 'Football'], 'Tiny Parkhurst': ['Computers', 'Farming'], 'Pearlie Moffet': ['Theatre', 'Cars', 'Videogames', 'Politics'], 'Cruz Perna': ['Farming', 'Internet'], 'Rebbecca Charlton': ['Beer', 'Theatre'], 'Marita Tegeler': ['Farming', 'Cats', 'Books', 'Internet'], 'Jarred Marrow': ['Programming', 'Theatre'], 'Lorean Simcox': ['Farming', 'Computers']}


In [ ]:
def find_final_rating(people_interests, new_brute_rating, coinciding_ratings=0):
    new_new_rating = dict(new_brute_rating)         #create a new dictionary for new ratings
    for key in people_interests.keys():             #for every person
        for interest in people_interests[key]:
            if interest in words_title:             #if an interest coincides with one from book's title
                coinciding_ratings += 1             #it is encounted
        new_new_rating[key] = new_new_rating[key]*0.2*coinciding_ratings    #after finding all similarities, the final rating is computed like in 3.6
    return new_new_rating

In [ ]:
new_new_rating = find_final_rating(people_interests, new_brute_rating)

#People sorted by final rating
print("People sorted by final rating:")
sorted_new_new_rating = sort_by_rating(new_new_rating)

for key, value in sorted_new_new_rating.items():
    if len(key) < 16:
        print("{}\t\t{}".format(key, value))
    else:
        print("{}\t{}".format(key, value))


People sorted by final rating:
Jarred Marrow		382.5
Rebbecca Charlton	291.06000000000006
Marita Tegeler		278.42499999999995
Tiny Parkhurst		236.60000000000002
Lorean Simcox		220.79999999999998
Sammie Womac		214.20000000000002
Angila Ellinger		207.90000000000003
Cruz Perna		165.375
Monet Mccoy		151.51500000000001
Alta Kennan		136.62
Elinore Orsborn		129.36
Lili Houghton		118.80000000000001
Otilia Laxson		88.69000000000001
Ellie Francese		88.01999999999998
Augustine Golub		87.57499999999999
Pearlie Moffet		83.84000000000002
Corrin Tally		77.17500000000003
Clarence Stalker	77.145
Leandro Eagan		67.83000000000001
Caleb Hobby		8.88


In [ ]:
print('People should be contacted for making the book a bestseller:')  
for i in range(5):
    print(list(sorted_new_new_rating.keys())[i])

People should be contacted for making the book a bestseller:
Jarred Marrow
Rebbecca Charlton
Marita Tegeler
Tiny Parkhurst
Lorean Simcox
